#### En este punto entiendo que solo se plantean las consultas dado que las tablas mencionadas en el ejercicio no existen, por lo que realizo el esquema de consultas con sqlite3

En primer lugar se importaría la librería y se generaría el conector.

In [ ]:
import sqlite3
import pandas as pd

#conexion
conn = sqlite3.connect("database.db")  # Reemplaza con tu archivo SQLite

query1 = """
SELECT 
    c.customer_id,
    c.name,
    SUM(od.quantity * od.price * (1 - od.discount)) AS total_income
FROM 
    customers c
JOIN 
    orders o ON c.customer_id = o.customer_id
JOIN 
    order_details od ON o.order_id = od.order_id
GROUP BY 
    c.customer_id, c.name
ORDER BY 
    total_income DESC;
"""
df_total_income = pd.read_sql_query(query1, conn)
print("Ingreso total por cliente:")
print(df_total_income)

Para esta primera consulta se hace uso de llaves foráneas y primarias en tabla de ordenes y clientes, realizando además una agrupación por cliente. Lo que permite obtener el ingreso total por cliente, el cual se ordena de manera descendente para obtener un ranking o top.

In [ ]:
# consulta 2: producto más vendido en cada región
query2 = """
WITH product_volume AS (
    SELECT 
        c.region,
        od.product_id,
        SUM(od.quantity * od.price) AS total_volume
    FROM 
        customers c
    JOIN 
        orders o ON c.customer_id = o.customer_id
    JOIN 
        order_details od ON o.order_id = od.order_id
    GROUP BY 
        c.region, od.product_id
)
SELECT 
    region,
    product_id,
    MAX(total_volume) AS max_volume
FROM 
    product_volume
GROUP BY 
    region;
"""
df_top_product_region = pd.read_sql_query(query2, conn)
print("\nProducto más vendido en cada región:")
print(df_top_product_region)

Para obtener el producto más vendido por región se deben crear dos tablas, la primera es tipo CTE usando with, donde primero a la tabla customer o cliente se le pega la región, el producto y se calcula el volumen, el cual se totaliza por region por producto en el group by. Finalmente, el resultado de esta tabla se le saca el máximo agrupado por región trayendo el id de producto, lo que en últimas permite obtener el producto con el mayor volumen de ventas por región.

In [ ]:
# consulta 3: ingreso promedio por cliente y región para cada mes
query3 = """
SELECT 
    c.region,
    strftime('%Y-%m', o.order_date) AS month,
    AVG(od.quantity * od.price * (1 - od.discount)) AS avg_income
FROM 
    customers c
JOIN 
    orders o ON c.customer_id = o.customer_id
JOIN 
    order_details od ON o.order_id = od.order_id
GROUP BY 
    c.region, month;
"""
df_avg_income = pd.read_sql_query(query3, conn)
print("\nIngreso promedio por cliente y región por mes:")
print(df_avg_income)

Para este requerimiento se parte  de un group by por región y mes, que ya permite asegurar la petición, además a la tabla customer se le pega la orden, así se calcula el ingreso promedio y adicional se formatea la fecha de orden al mes para obtener el resultado final

In [ ]:
# consulta 4: top 5 clientes con más ingresos en el último año
query4 = """
WITH last_year_orders AS (
    SELECT 
        *
    FROM 
        orders
    WHERE 
        order_date >= date('now', '-1 year')
)
SELECT 
    c.customer_id,
    c.name,
    SUM(od.quantity * od.price * (1 - od.discount)) AS total_income,
    COUNT(DISTINCT o.order_id) AS total_orders
FROM 
    customers c
JOIN 
    last_year_orders o ON c.customer_id = o.customer_id
JOIN 
    order_details od ON o.order_id = od.order_id
GROUP BY 
    c.customer_id, c.name
ORDER BY 
    total_income DESC
LIMIT 5;
"""
df_top_customers = pd.read_sql_query(query4, conn)
print("\nTop 5 clientes con más ingresos en el último año:")
print(df_top_customers)

conn.close()

Para esta consulta vuelvo a hacer uso del with, ¿Por qué? pporque se requieren tener tablas intermedias auxiliares para el calculo final. Con last_year_orders obtengo el año más reciente o el último año, después hago los cálculos de ingreso y volumen con costumer y ordenes y el ranking de 5 con más ingresos pues se obtiene simpemente con el order by limitado a 5 o truncado a 5. Finalizo cerrando la conexión de la API de sqlite. Este tipo de ejercicios también se pueden realizar con pysql.